<a href="https://colab.research.google.com/github/Rafathebull/Rafathebull/blob/main/Task1_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import torch
from torch.utils.data import DataLoader, TensorDataset

task_1a_dataframe = pd.read_csv('/content/drive/MyDrive/task_1a_dataset.csv')
def data_preprocessing(task_1a_dataframe):
    # Create a copy of the original dataframe to avoid modifying the original data
    encoded_dataframe = task_1a_dataframe.copy()

    # Initialize a LabelEncoder for encoding categorical features
    label_encoder = LabelEncoder()

    # Iterate through columns in the dataframe
    for column in encoded_dataframe.columns:
        # Check if the column data type is object (textual)
        if encoded_dataframe[column].dtype == 'object':
            # Encode the textual feature into numerical labels
            encoded_dataframe[column] = label_encoder.fit_transform(encoded_dataframe[column])

    return encoded_dataframe

encoded_dataframe = data_preprocessing(task_1a_dataframe)

def identify_features_and_targets(encoded_dataframe):
    features = ['Education','ExperienceInCurrentDomain','PaymentTier','Age','City','Gender']
    target = 'LeaveOrNot'
    features_and_targets = [features, target]

    return features_and_targets
features_and_targets = identify_features_and_targets(encoded_dataframe)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
def load_as_tensors(features_and_targets):
    # Extract features and target label from the input list
    features, target = features_and_targets

    # Assuming you have your data loaded and split into X_train, X_test, y_train, y_test
    # Convert NumPy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(features, dtype=torch.float32)
    y_train_tensor = torch.tensor(target, dtype=torch.float32)

    # Create a TensorDataset for training data
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

    # Create a DataLoader for training data to make it iterable
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Return the tensors and iterable training data
    tensors_and_iterable_training_data = [X_train_tensor, y_train_tensor, train_loader]

    return tensors_and_iterable_training_data

    features = ['Education','ExperienceInCurrentDomain','PaymentTier','Age','City','Gender']
    target = 'LeaveOrNot'
    features_and_targets = [features, target]
    tensors_and_iterable_training_data = load_as_tensors(features_and_targets)

nn model

In [56]:
import torch
import torch.nn as nn

class Salary_Predictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Salary_Predictor, self).__init__()

        # Define the layers of the neural network
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        '''
        Define the activation functions and how data flows through the layers
        '''
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x

# Example usage:
# Assuming input size is 3, hidden size is 64, and output size is 1
model = Salary_Predictor(input_size=6, hidden_size=64, output_size=1)

In [57]:
def model_loss_function():
    loss_function = nn.MSELoss()

    return loss_function

# Example usage:
loss_function = model_loss_function()

import torch.optim as optim

def model_optimizer(model):
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    return optimizer

# Example usage:
# Assuming you have already created an instance of the 'Salary_Predictor' model
model = Salary_Predictor(input_size=6, hidden_size=64, output_size=1)
optimizer = model_optimizer(model)


In [58]:
def model_number_of_epochs():
    number_of_epochs = 100
    return number_of_epochs

# Example usage:
number_of_epochs = model_number_of_epochs()



In [59]:
def training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer):
    X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader = tensors_and_iterable_training_data

    for epoch in range(number_of_epochs):
        model.train()  # Set the model to training mode
        total_loss = 0.0

        for batch_X, batch_y in train_loader:
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_X)

            # Calculate the loss
            loss = loss_function(outputs, batch_y)

            # Backpropagation and optimization
            loss.backward()
            optimizer.step()

            # Update the total loss
            total_loss += loss.item()

        # Print the average loss for this epoch
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{number_of_epochs}], Loss: {avg_loss:.4f}')

    # Return the trained model
    return model

# Example usage:
# Assuming you have already defined the required parameters
    tensors_and_iterable_training_data = [X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader]

    return tensors_and_iterable_training_data
    features = ['Education','ExperienceInCurrentDomain','PaymentTier','Age','City','Gender']
    target = 'LeaveOrNot'
    features_and_targets = [features, target]
    tensors_and_iterable_training_data = load_as_tensors(features_and_targets)
    trained_model = training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer)


In [60]:
def validation_function(trained_model, tensors_and_iterable_training_data):
    X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader = tensors_and_iterable_training_data

    trained_model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient tracking for evaluation
        for batch_X, batch_y in train_loader:
            # Forward pass
            outputs = trained_model(batch_X)

            # Convert the model's predictions to class labels (e.g., 0 or 1)
            predicted_labels = (outputs >= 0.5).float()

            # Count correct predictions
            correct_predictions += (predicted_labels == batch_y).sum().item()
            total_samples += batch_y.size(0)
    # Calculate accuracy
    model_accuracy = (correct_predictions / total_samples)
    return model_accuracy


In [ ]:
# Define and load your data, model, loss function, and optimizer
features_and_targets = identify_features_and_targets(encoded_dataframe)
tensors_and_iterable_training_data = load_as_tensors(features_and_targets)
model = Salary_Predictor(input_size=3, hidden_size=64, output_size=1)
loss_function = model_loss_function()
optimizer = model_optimizer(model)
number_of_epochs = model_number_of_epochs()

# Train the model
trained_model = training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer)

# Validate the model
model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)
print(f'Model Accuracy on Validation Dataset: {model_accuracy:.2f}%')


ValueError: ignored

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load your data from a CSV file or another source into a DataFrame
data = pd.read_csv('/content/drive/MyDrive/task_1a_dataset.csv')

# Define your features and target variable
features = ['Education','ExperienceInCurrentDomain','PaymentTier','Age','City','Gender']
target = 'LeaveOrNot'

# Check data types of columns
non_numeric_columns = data.select_dtypes(exclude=['float', 'int']).columns
batch_size = 100
# Handle non-numeric columns (e.g., label encoding)
label_encoders = {}
for column in non_numeric_columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

# Split your data into training and testing sets
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert your training and testing data into tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Create a TensorDataset for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Create a DataLoader for training data to make it iterable
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Return the tensors and iterable training data
tensors_and_iterable_training_data = [X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader]


In [62]:
# Define and load your data, model, loss function, and optimizer
features = ['Education','ExperienceInCurrentDomain','PaymentTier','Age','City','Gender']
target = 'LeaveOrNot'
features_and_targets = [features, target]

# Assuming you have your data loaded and preprocessed, split into X_train, X_test, y_train, y_test
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Define the batch size
batch_size = 100  # You can adjust this value based on your needs

# Create a TensorDataset for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Create a DataLoader for training data to make it iterable
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Return the tensors and iterable training data
tensors_and_iterable_training_data = [X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader]

model = Salary_Predictor(input_size=len(features), hidden_size=64, output_size=1)
loss_function = model_loss_function()
optimizer = model_optimizer(model)
number_of_epochs = model_number_of_epochs()

# Train the model
trained_model = training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer)
print("Model training complete.")

# Validate the model
model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)
print(f'Model Accuracy on Validation Dataset: {model_accuracy:.2f}%')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([6, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/100], Loss: 0.2371
Epoch [2/100], Loss: 0.2283
Epoch [3/100], Loss: 0.2273
Epoch [4/100], Loss: 0.2258
Epoch [5/100], Loss: 0.2227
Epoch [6/100], Loss: 0.2255
Epoch [7/100], Loss: 0.2243
Epoch [8/100], Loss: 0.2239
Epoch [9/100], Loss: 0.2265
Epoch [10/100], Loss: 0.2298
Epoch [11/100], Loss: 0.2260
Epoch [12/100], Loss: 0.2249
Epoch [13/100], Loss: 0.2241
Epoch [14/100], Loss: 0.2241
Epoch [15/100], Loss: 0.2248
Epoch [16/100], Loss: 0.2259
Epoch [17/100], Loss: 0.2239
Epoch [18/100], Loss: 0.2262
Epoch [19/100], Loss: 0.2274
Epoch [20/100], Loss: 0.2228
Epoch [21/100], Loss: 0.2256
Epoch [22/100], Loss: 0.2257
Epoch [23/100], Loss: 0.2254
Epoch [24/100], Loss: 0.2270
Epoch [25/100], Loss: 0.2228
Epoch [26/100], Loss: 0.2282
Epoch [27/100], Loss: 0.2258
Epoch [28/100], Loss: 0.2269
Epoch [29/100], Loss: 0.2236
Epoch [30/100], Loss: 0.2289
Epoch [31/100], Loss: 0.2296
Epoch [32/100], Loss: 0.2252
Epoch [33/100], Loss: 0.2258
Epoch [34/100], Loss: 0.2266
Epoch [35/100], Loss: 0

In [64]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Define your model, loss function, optimizer, and other parameters
# (I'll assume you have these defined as functions)

# Define and load your data, model, loss function, and optimizer
features = ['Education', 'ExperienceInCurrentDomain', 'PaymentTier', 'Age', 'City', 'Gender']
target = 'LeaveOrNot'
features_and_target = [features, target]

# Assuming you have your data loaded and preprocessed, split into X_train, X_test, y_train, y_test
X_train_tensor = torch.tensor(X_train[features].values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train[target].values, dtype=torch.float32)  # Reshape target
X_test_tensor = torch.tensor(X_test[features].values.reshape(-1, 1), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test[target].values.reshape(-1, 1), dtype=torch.float32)  # Reshape target

# Define the batch size
batch_size = 100  # You can adjust this value based on your needs

# Create a TensorDataset for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Create a DataLoader for training data to make it iterable
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Return the tensors and iterable training data
tensors_and_iterable_training_data = [X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, train_loader]

model = Salary_Predictor(input_size=len(features), hidden_size=64, output_size=1)
loss_function = model_loss_function()
optimizer = model_optimizer(model)
number_of_epochs = model_number_of_epochs()

# Train the model
trained_model = training_function(model, number_of_epochs, tensors_and_iterable_training_data, loss_function, optimizer)
print("Model training complete.")

# Validate the model
model_accuracy = validation_function(trained_model, tensors_and_iterable_training_data)
print(f'Model Accuracy on Validation Dataset: {model_accuracy:.2f}%')

KeyError: ignored